In [ ]:
import zipfile
import os


zip_path = "/content/Models.zip"
extract_path = "/content/unzipped"

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Files extracted successfully to:", extract_path)
print(os.listdir(extract_path))


Files extracted successfully to: /content/unzipped
['Models']


In [ ]:
import tensorflow as tf
print("GPU Available:", tf.config.list_physical_devices('GPU'))
!nvidia-smi


GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Thu Mar  6 16:27:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |               

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.set_visible_devices(gpus[0], 'GPU')
        print("GPU is enabled.")
    except RuntimeError as e:
        print(e)


GPU is enabled.


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(" GPU acceleration enabled!")
    except RuntimeError as e:
        print(e)

path = r"/content/unzipped/Models/Hair Diseases - Final"
train_dir = os.path.join(path,"train")
val_dir = os.path.join(path,"val")
test_dir = os.path.join(path,"test")

train_datagen = ImageDataGenerator(
    rescale=1./255, rotation_range=15, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.1, zoom_range=0.1,
    horizontal_flip=True, fill_mode="nearest"
)
val_test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 64
image_size = (224, 224)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical'
)
val_generator = val_test_datagen.flow_from_directory(
    val_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical'
)
test_generator = val_test_datagen.flow_from_directory(
    test_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical'
)


AUTOTUNE = tf.data.AUTOTUNE

def preprocess(dataset):
    return tf.data.Dataset.from_generator(
        lambda: dataset,
        output_signature=(
            tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, len(train_generator.class_indices)), dtype=tf.float32)
        )
    ).cache().prefetch(buffer_size=AUTOTUNE)

train_dataset = preprocess(train_generator)
val_dataset = preprocess(val_generator)
test_dataset = preprocess(test_generator)


print(f"Train samples: {train_generator.samples}")
print(f"Validation samples: {val_generator.samples}")
print(f"Test samples: {test_generator.samples}")


 GPU acceleration enabled!
Found 9600 images belonging to 10 classes.
Found 1200 images belonging to 10 classes.
Found 1200 images belonging to 10 classes.
Train samples: 9600
Validation samples: 1200
Test samples: 1200


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

path = r"/content/unzipped/Models/Hair Diseases - Final"
train_dir = os.path.join(path, "train")
val_dir = os.path.join(path, "val")
test_dir = os.path.join(path, "test")

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    channel_shift_range=30.0,
    fill_mode="nearest"
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 64
image_size = (224, 224)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical')

base_model = EfficientNetV2S(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers[:int(len(base_model.layers) * 0.5)]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.25)(x)
output_layer = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output_layer)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=["accuracy"])

callbacks = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_loss"),
]

history = model.fit(train_generator, validation_data=val_generator, epochs=10, batch_size=batch_size, callbacks=callbacks)

for layer in base_model.layers:
    layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss="categorical_crossentropy", metrics=["accuracy"])

history_finetune = model.fit(train_generator, validation_data=val_generator, epochs=15, batch_size=batch_size, callbacks=callbacks)

model.save("efficientnetv2_hair_disease_optimized.h5")

test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc * 100:.2f}%")


Found 9600 images belonging to 10 classes.
Found 1200 images belonging to 10 classes.
Found 1200 images belonging to 10 classes.
Epoch 1/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1693 - loss: 2.5036

150/150 ━━━━━━━━━━━━━━━━━━━━ 275s 1s/step - accuracy: 0.1695 - loss: 2.5024 - val_accuracy: 0.4008 - val_loss: 1.7958
Epoch 2/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2713 - loss: 2.0540

150/150 ━━━━━━━━━━━━━━━━━━━━ 158s 1s/step - accuracy: 0.2714 - loss: 2.0537 - val_accuracy: 0.4517 - val_loss: 1.5835
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3021 - loss: 1.9277

150/150 ━━━━━━━━━━━━━━━━━━━━ 159s 1s/step - accuracy: 0.3022 - loss: 1.9276 - val_accuracy: 0.4875 - val_loss: 1.4662
Epoch 4/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3250 - loss: 1.8620

150/150 ━━━━━━━━━━━━━━━━━━━━ 202s 1s/step - accuracy: 0.3251 - loss: 1.8618 - val_accuracy: 0.5000 - val_loss: 1.3691
Epoch 5/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3490 - loss: 1.7848

...

150/150 ━━━━━━━━━━━━━━━━━━━━ 179s 1s/step - accuracy: 0.9641 - loss: 0.1286 - val_accuracy: 0.9958 - val_loss: 0.0144
Epoch 12/15
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9676 - loss: 0.1160

150/150 ━━━━━━━━━━━━━━━━━━━━ 179s 1s/step - accuracy: 0.9677 - loss: 0.1159 - val_accuracy: 0.9992 - val_loss: 0.0078
Epoch 13/15
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9757 - loss: 0.0870

150/150 ━━━━━━━━━━━━━━━━━━━━ 180s 1s/step - accuracy: 0.9757 - loss: 0.0870 - val_accuracy: 0.9975 - val_loss: 0.0078
Epoch 14/15
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9838 - loss: 0.0641

150/150 ━━━━━━━━━━━━━━━━━━━━ 201s 1s/step - accuracy: 0.9838 - loss: 0.0640 - val_accuracy: 0.9992 - val_loss: 0.0038
Epoch 15/15
150/150 ━━━━━━━━━━━━━━━━━━━━ 171s 1s/step - accuracy: 0.9824 - loss: 0.0578 - val_accuracy: 0.9983 - val_loss: 0.0038


19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 238ms/step - accuracy: 1.0000 - loss: 0.0017
Test Accuracy: 100.00%
